In [34]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
import re
import sys


def cleanText(tweet):
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    tweet = ' '.join(re.sub("[\|\.\,\"\'\!\?\:\;\-\=]", " ", tweet).split())
    tweet = tweet.lower()
    
    return tweet


df = pd.read_csv('data/data_tweeter.csv', sep=";")
l = []
for i,j in zip(df['texto'], df['class']):
    t = cleanText(i)
    l.append([t, j])


[['rapidez da peste música ednaldo pereira', 'musica'], ['god is good dance por ednaldo pereira', 'musica'], ['ednaldo pereira skate montagem por legal ednaldo pereira', 'musica'], ['ednaldo pereira vermelho brasil montagem por _foda ednaldo pereira', 'musica'], ['bandeira de fã ruiva montagem por ednaldo pereira', 'musica'], ['um bom modo de enfrentar a contaminação pelo preconceito é oferecer informação (via _ombudsman)', 'sociedade'], ['ednaldo pereira desenho pequeno legal por valeu ednaldo pereira', 'musica'], ['desenho ednaldo pereira bebê estranho por _is_onnn valeu ednaldo pereira', 'musica'], ['as histórias reveladas pelos documentos que a polícia secreta da alemanha oriental não conseguiu destruir', 'sociedade'], ['cotação usada passará a ser a do dia da compra e não mais a do fechamento da fatura', 'economia'], ['nunca pensei em entrar na política tinha asco diz eduardo bolsonaro', 'politica'], ['indicadores de fevereiro serão divulgados nesta semana', 'saude'], ['pms resgat

In [36]:
columns = ['texto', 'class']
rows = l

# rows = [['This is my book', 'stmt'], 
#         ['They are novels', 'stmt'],
#         ['have you read this book', 'question'],
#         ['who is the author', 'question'],
#         ['what are the characters', 'question'],
#         ['This is how I bought the book', 'stmt'],
#         ['I like fictions', 'stmt'],
#         ['what is your favorite book', 'question']]

training_data = pd.DataFrame(rows, columns=columns)



#CountVect
stmt_docs = [row['texto'] for index,row in training_data.iterrows() if row['class'] == 'politica']

vec_s = CountVectorizer()
X_s = vec_s.fit_transform(stmt_docs)
tdm_s = pd.DataFrame(X_s.toarray(), columns=vec_s.get_feature_names())

q_docs = [row['texto'] for index,row in training_data.iterrows() if row['class'] == 'musica']

vec_q = CountVectorizer()
X_q = vec_q.fit_transform(q_docs)
tdm_q = pd.DataFrame(X_q.toarray(), columns=vec_q.get_feature_names())


,_foda,_is_onnn,bandeira,bebê,brasil,cancela,da,dance,de,desenho,...,peste,por,rapidez,ruiva,show,skate,turnê,um,valeu,vermelho
0,0,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
6,0,1,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
7,0,0,0,0,0,1,1,0,0,0,...,0,1,0,0,1,0,1,1,0,0


In [32]:
#Frequency
word_list_s = vec_s.get_feature_names();    
count_list_s = X_s.toarray().sum(axis=0) 
freq_s = dict(zip(word_list_s,count_list_s))

word_list_q = vec_q.get_feature_names();    
count_list_q = X_q.toarray().sum(axis=0) 
freq_q = dict(zip(word_list_q,count_list_q))

#Prob
prob_s = []
for word, count in zip(word_list_s, count_list_s):
    prob_s.append(count/len(word_list_s))

probs = dict(zip(word_list_s, prob_s))

prob_q = []
for word, count in zip(word_list_q, count_list_q):
    prob_q.append(count/len(word_list_q))
    
probq = dict(zip(word_list_q, prob_q))


docs = [row['texto'] for index,row in training_data.iterrows()]

vec = CountVectorizer()
X = vec.fit_transform(docs)

total_features = len(vec.get_feature_names())

total_cnts_features_s = count_list_s.sum(axis=0)
total_cnts_features_q = count_list_q.sum(axis=0)



In [ ]:

new_sentence = 'Ednaldo'
new_word_list = word_tokenize(new_sentence)

prob_s_with_ls = []
for word in new_word_list:
    if word in freq_s.keys():
        count = freq_s[word]
    else:
        count = 0
    prob_s_with_ls.append((count + 1)/(total_cnts_features_s + total_features))
    
r_s = dict(zip(new_word_list,prob_s_with_ls))

prob_q_with_ls = []
for word in new_word_list:
    if word in freq_q.keys():
        count = freq_q[word]
    else:
        count = 0
    prob_q_with_ls.append((count + 1)/(total_cnts_features_q + total_features))
    
r_q = dict(zip(new_word_list,prob_q_with_ls))

print(r_q)

# df.drop(['Unnamed: 0'], inplace=True, axis=1)


# document = ["This is the most beautiful place in the world.", "This man has more skills to show in cricket than any other game.", "Hi there! how was your ladakh trip last month?", "There was a player who had scored 200+ runs in single cricket innings in his career.", "I have got the opportunity to travel to Paris next year for my internship.", "May be he is better than you in batting but you are much better than him in bowling.", "That was really a great day for me when I was there at Lavasa for the whole night.", "That’s exactly I wanted to become, a highest ratting batsmen ever with top scores.", "Does it really matter wether you go to Thailand or Goa, its just you have spend your holidays.", "Why don’t you go to Switzerland next year for your 25th Wedding anniversary?", "Travel is fatal to prejudice, bigotry, and narrow mindedness., and many of our people need it sorely on these accounts.", "Stop worrying about the potholes in the road and enjoy the journey.", "No cricket team in the world depends on one or two players. The team always plays to win.", "Cricket is a team game. If you want fame for yourself, go play an individual game.", "Because in the end, you won’t remember the time you spent working in the office or mowing your lawn. Climb that goddamn mountain.", "Isn’t cricket supposed to be a team sport? I feel people should decide first whether cricket is a team game or an individual sport."]

# document =  df['texto']
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.cluster import KMeans

# vectorizer = TfidfVectorizer(stop_words='english')
# X = vectorizer.fit_transform(document)


# true_k = 2
# model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
# model.fit(X)

# order_centroids = model.cluster_centers_.argsort()[:, ::-1]
# terms = vectorizer.get_feature_names()

# for i in range(true_k):
#     print("Cluster %d:" % i),
#     for ind in order_centroids[i, :10]:
#         print(" %s" % terms[ind])